In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

import re

In [2]:
# patients = pd.read_csv('../data/pts.csv')

# patients.columns = [x.lower().replace(' ', '_') for x in patients.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# patients.to_sql('patients', db, if_exists = 'append', index = False) 

299712

In [3]:
# admission = pd.read_csv('../data/hos/admissions.csv')

# admission.columns = [x.lower().replace(' ', '_') for x in admission.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# admission.to_sql('admission', db, if_exists = 'append', index = False) 

431231

In [4]:
# diagicd = pd.read_csv('../data/diag_icd.csv')

# diagicd.columns = [x.lower().replace(' ', '_') for x in diagicd.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# diagicd.to_sql('diagicd', db, if_exists = 'append', index = False) 

4756326

In [ ]:
# subject_id	hadm_id patient

In [31]:
dis.loc[j, 'text']

' \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___.\n \nChief Complaint:\nDyspnea and chest tightness\n \nMajor Surgical or Invasive Procedure:\n___ Pharmacologic nuclear stress test\n \nHistory of Present Illness:\n___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and across lower \nrib c

In [103]:
text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']


tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])



# for i in range (len(text_subheading)-1):
#     text_parsed =tt1.pop()

# print(i, text_parsed, len('text_subheading'))

In [104]:
tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
dic = {}
tt1.pop(0)

while len(tt1) > 0:
    
    key = tt1.pop(0)
    value = tt1.pop(0)

    if key in text_subheading:
        dic[key] = value

In [105]:
dic

{'Chief Complaint': 'Dyspnea and chest tightness\n ',
 'Major Surgical or Invasive Procedure': '___ Pharmacologic nuclear stress test\n ',
 'History of Present Illness': '___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and across lower \nrib cage, persistent since onset, no radiation to the \nshoulders/jaw/back, no diaphoreses. Worsens with activity, \nimproves somewhat with rest. Patient does not it feels like \nother episodes when she then required her stent placement. Took \n

In [95]:
tt1

[' \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___.\n ',
 'Chief Complaint',
 'Dyspnea and chest tightness\n ',
 'Major Surgical or Invasive Procedure',
 '___ Pharmacologic nuclear stress test\n ',
 'History of Present Illness',
 '___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and acro

In [76]:
# tt1.index('Discharge Instructions')
# i = 'Chief Complaint'
for i in range (len(list_subheading)-1):
#     index = tt1.index(i)
    text_parsed =tt1.pop(i+1)
    
    text_parsed

In [116]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_e78 = pd.read_csv('../data/icd/e78.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_e78, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [119]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_i16 = pd.read_csv('../data/icd/i16.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_i16, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [121]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_y92 = pd.read_csv('../data/icd/y92.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_y92, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [123]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_e11 = pd.read_csv('../data/icd/e11.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_e11, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [ ]:
df_text.to_sql('text', db, if_exists = 'append', index = False)

In [12]:
df_text

,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,family_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,18442315,26150389,I50,"Congestive heart failure, unspecified",shortness of breath \n,None. \n\n,___ y/o female with a past medical history of ...,PMH: \n1. Hypertension.\n2. Osteoporosis.\n...,___,non-contributory\n\n,...,NaN,ADMISSION LABS:\n================\n___ 01:02PM...,___ y/o female with a past medical history of ...,The Preadmission Medication list is accurate a...,1. Rivaroxaban 15 mg PO DAILY \nTake with the ...,Extended Care\n,Atrial Fibrillation\nCOPD exacerbation\nUrinar...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt has been a pleasure taking...",___\n
1,18442315,26150389,I50,Acute systolic heart failure,shortness of breath \n,None. \n\n,___ y/o female with a past medical history of ...,PMH: \n1. Hypertension.\n2. Osteoporosis.\n...,___,non-contributory\n\n,...,NaN,ADMISSION LABS:\n================\n___ 01:02PM...,___ y/o female with a past medical history of ...,The Preadmission Medication list is accurate a...,1. Rivaroxaban 15 mg PO DAILY \nTake with the ...,Extended Care\n,Atrial Fibrillation\nCOPD exacerbation\nUrinar...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt has been a pleasure taking...",___\n
2,18442432,27855871,I50,Unspecified diastolic (congestive) heart failure,Bright red blood per rectum\n,None\n,This is a ___ woman with a history of Afib s/p...,___ esophagus \nS/P Mechanical aortic valve \n...,___,Her father died at age ___- MI. ___ mother die...,...,NaN,ADMISSION LABS\n___ 05:05PM BLOOD WBC-7.7 RBC-...,NaN,The Preadmission Medication list is accurate a...,1. Enoxaparin Sodium 70 mg SC Q12H \nStart: T...,Home\n,Lower gastrointestinal bleeding\nColon polyps\...,Mental Status: Clear and coherent.\nLevel of C...,You were hospitalized for bleeding after colon...,___\n
3,18443495,24686393,I50,Unspecified diastolic (congestive) heart failure,Seizures\n,intubation\n,Eu Critical ___ aka ___ (MRN ___ is a ___ with...,# Type 1 diabetes diagnosed at age ___. H/o me...,___,Her father has AIDS. Her mother has diabetes a...,...,NaN,===ADMISSION LABS===\n___ 04:19PM BLOOD WBC-13...,"#SEIZURE\nAdmitted to ICU, intubated/sedated f...",The Preadmission Medication list is accurate a...,1. Divalproex (DELayed Release) 1000 mg PO Q8...,Home With Service\n,status epilepticus\nend stage renal disease\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Ms. ___, \nYou were admitted with seizures. To...",___\n
4,18443821,20382977,I50,Acute on chronic diastolic (congestive) heart ...,"Dyspnea, leg swelling, chest pain\n",none\n\n,___ with history of CAD (s/p 1x stent in ___ i...,- CAD s/p 1x stent (placed in ___ in ___ \n- ...,___,___ family history of early MI or history of h...,...,NaN,ADMISSION LABS:\n==============\n___ 09:46PM B...,"___ with CAD s/p PCI ___ (unknown details), CK...",Patient does not know his medications. On adm...,1. amLODIPine 10 mg PO DAILY \nRX *amlodipine...,Home\n,Primary Diagnosis:\nAcute exacerbation of dias...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Mr. ___,\n\nYou were admitted to the hosp...",___\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2422,18731959,21511199,I50,Acute on chronic systolic (congestive) heart f...,Elective admission for percutaneous coronary i...,___ Cardiac catheterization \n,___ year old woman with of lung CA (small cell...,1. Lung cancer diagnosed ___: s/p chemotherapy...,___,"Brother alive and well, age ___. Mother health...",...,NaN,===============,Ms. ___ is a ___ with a background history of ...,1. Anastrozole 1 mg PO DAILY \n2. Aspirin 81 m...,1. Azithromycin 250 mg PO DAILY Duration: 4 D...,Home\n,

In [42]:
#checking the tables in the sqlite_db

db = sqlite3.connect('../data/ehr.sqlite')
db.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""").fetchall()

[('patients',), ('admission',), ('diagicd',), ('text',), ('text_ic50',)]

In [22]:
# query = """
# SELECT * 
# FROM text ```````````
# WHERE subject_id = '10000980'
# """
query = """
SELECT COUNT (*) 
FROM text
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,COUNT (*)
0,0


In [11]:
query = """
SELECT * 
FROM text 
WHERE icd = 'I50'
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,family_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,10000980,29654838,I50,"Congestive heart failure, unspecified",Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,Denies cardiac family history. Family hx of DM...,...,None,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
1,10000980,29654838,I50,Acute on chronic diastolic heart failure,Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,Denies cardiac family history. Family hx of DM...,...,None,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
2,10000980,26913865,I50,Acute on chronic systolic heart failure,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,"Niece had some sort of cancer. Otherwise, no f...",...,None,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
3,10000980,26913865,I50,"Congestive heart failure, unspecified",dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,"Niece had some sort of cancer. Otherwise, no f...",...,None,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
4,10000980,24947999,I50,Acute on chronic systolic heart failure,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,Niece had some sort of cancer. Father died in ...,...,None,==== ADMISSION LABS ====\n___ 04:00PM BLOOD WB...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68489,18439835,22458701,I50,Chronic diastolic heart failure,Pt presenting for initiation of dialysis\n,None\n\n,Mrs. ___ is a pleasant ___ yo F w/ stage V CKD...,- Type 2 Diabetes \n- Coronary artery disease...,___,Mother with heart disease in ___ and cervical ...,...,None,On admission\n___ 02

In [47]:
#As the database crashed in the middle of adding IC50 rows we wanted to delete and restore the table 
#so that when adding baack there is no duplication of rows.
# query1 = """
# DELETE FROM text 
# WHERE icd = 'I50';
# """

# query2 = """
# COMMIT;
# """

#maximum allowed pages n db
query3= """
PRAGMA max_page_count;  
"""

# number of pages we have in db
query4= """
PRAGMA page_count;
"""

query5= """
PRAGMA temp_store_directory;
"""

query6 = """
DROP TABLE text_ic50;
"""
query7 = """
DROP TABLE text;
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
#     db.execute(query6)
    db.execute(query7)
#    page_count = db.execute(query4)
#     directory = db.execute(query5)
# page_count.fetchall()
# directory.fetchall()


#1073741823 max_count, page#we have (1046870,)

In [48]:
#checking the tables in the sqlite_db

db = sqlite3.connect('../data/ehr.sqlite')
db.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""").fetchall()

[('patients',), ('admission',), ('diagicd',), ('ictarget',)]

In [5]:
# db = sqlite3.connect('../data/ehr.sqlite')

df = []

text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions', '$']

# df_i50 = pd.read_csv('../data/icd/i50.csv')
# df_e11 = pd.read_csv('../data/icd/e11.csv')
# df_e78 = pd.read_csv('../data/icd/e78.csv')
# df_n17 = pd.read_csv('../data/icd/n17.csv')
# df_z85 = pd.read_csv('../data/icd/z85.csv')
# df_f32 = pd.read_csv('../data/icd/f32.csv')
# df_i16 = pd.read_csv('../data/icd/i16.csv')
# df_y92 = pd.read_csv('../data/icd/y92.csv')
df_e87 = pd.read_csv('../data/icd/e87.csv')

for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
    dis = chunk.merge(df_e87, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
    dic_list = []
    for j in range (len(dis)):
        tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
        dic = {'subject_id': (dis.loc[j, 'subject_id']),
               'hadm_id' : (dis.loc[j, 'hadm_id']),
              'icd' : (dis.loc[j, 'icd']),
              'long_title' : (dis.loc[j, 'long_title'])}
        tt1.pop(0)

        while len(tt1) > 0:
    
            key = tt1.pop(0)
            value = tt1.pop(0)

            if key in text_subheading:
                dic[key] = value

      
 
        dic_list.append(dic)


    df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
     'Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
     'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
     'History of Present Illness': 'history_present_illness',
     'Past Medical History': 'past_medcal_history',
     'Social History':'social_history',
     'Family History':'family_history',
     'Physical Exam':'physical_exam',
     'Discharge':'discharge',
     'Pertinent Results': 'pertinent_results',
     'Brief Hospital Course': 'hospital_course',
     'Medications on Admission':'medication_on_admission',
     'Discharge Medications':'medication_on_discharge',
     'Discharge Disposition': 'discharge_disposition',
     'Discharge Diagnosis': 'discharge_diagnosis',
     'Discharge Condition':'discharge_condition',
     'Discharge Instructions':'discharge_instruction',
     'Followup Instructions':'followup_instruction'})
   


        # filter it to the subjectId and hadm_id
    df.append(df_text)
df_icd = pd.concat(df)
# df_icd.to_csv('../data/text_ice11.csv')
# df_icd.to_csv('../data/text_ici50.csv')
# df_icd.to_csv('../data/text_ice78.csv')
# df_icd.to_csv('../data/text_icn17.csv')
# df_icd.to_csv('../data/text_icf32.csv')#add .csv
# df_icd.to_csv('../data/text_icz85.csv')
# df_icd.to_csv('../data/text_ici16.csv')
# df_icd.to_csv('../data/text_icy92.csv')
# df_icd.to_csv('../data/text_ice87.csv')
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

    

0it [00:00, ?it/s]

In [6]:
text_e11 = pd.read_csv("../data/text_ice11.csv")
text_e11.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000980,29654838,E11,Diabetes mellitus without mention of complicat...,Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,NaN,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
1,1,10000980,26913865,E11,Diabetes mellitus without mention of complicat...,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,NaN,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
2,2,10000980,24947999,E11,Diabetes mellitus without mention of complicat...,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,==== ADMISSION LABS ====\n___ 04:00PM BLOOD WB...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n
3,3,10000980,25242409,E11,"Diabetes with renal manifestations, type II or...",DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,ADMISSION LABS:\n\n___ 02:45PM BLOOD WBC-5.1 R...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n
4,4,10000980,25242409,E11,"Diabetes with neurological manifestations, typ...",DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,ADMISSION LABS:\n\n___ 02:45PM BLOOD WBC-5.1 R...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n


In [7]:
text_i50 = pd.read_csv("../data/text_ici50.csv")
text_i50.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000980,29654838,I50,"Congestive heart failure, unspecified",Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,NaN,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
1,1,10000980,29654838,I50,Acute on chronic diastolic heart failure,Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,NaN,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
2,2,10000980,26913865,I50,Acute on chronic systolic heart failure,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,NaN,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
3,3,10000980,26913865,I50,"Congestive heart failure, unspecified",dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,NaN,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
4,4,10000980,24947999,I50,Acute on chronic systolic heart failure,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,==== ADMISSION LABS ====\n___ 04:00PM BLOOD WB...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n


In [8]:
text_e78 = pd.read_csv("../data/text_ice78.csv")
text_e78.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000084,23052089,E78,"Hyperlipidemia, unspecified",Visual hallucinations\n,N/A\n\n,"___ male with ___ disease, dyslipidemia, and a...",___ disease\n___ Body Dementia \ndyslipidemia ...,___,...,NaN,ADMISSION LABS:\n===============\n___ 10:30PM ...,NaN,The Preadmission Medication list may be inaccu...,1. QUEtiapine Fumarate 25 mg PO QHS \nRX *que...,Home With Service\n,___ Dementia\n,Mental Status: Confused - sometimes.\nLevel of...,"Dear Mr. ___, \n\nIt was a privilege caring fo...",___\n
1,1,10000117,27988844,E78,"Pure hypercholesterolemia, unspecified",Left hip pain \n,"Status post left CRPP ___, ___\n\n",REASON FOR CONSULT: Femur fracture\n\nHPI: ___...,- GERD \n - Hypercholesterolemia \n - Kidney...,___,...,NaN,NaN,The patient presented to the emergency departm...,The Preadmission Medication list is accurate a...,1. Acetaminophen 1000 mg PO Q6H:PRN Pain - Mi...,Home With Service\n,Left valgus impacted femoral neck fracture\n\n,"AVSS\nNAD, A&Ox3\nLLE: Incision well approxima...",INSTRUCTIONS AFTER ORTHOPAEDIC SURGERY:\n\n- Y...,___\n
2,2,10000764,27897940,E78,Other and unspecified hyperlipidemia,Epistaxis\n,None\n\n,Mr. ___ is an ___ with history of AAA s/p repa...,MI after AAA repair when he was ___ y/o\nHTN\n...,___,...,NaN,ADMISSION:\n___ 08:15PM BLOOD WBC-11.3* RBC-4....,Mr. ___ is an ___ with history of AAA s/p repa...,The Preadmission Medication list is accurate a...,1. Clopidogrel 75 mg PO DAILY \n2. Acetaminoph...,Home With Service\n,Nasal fracture\nEpistaxis\nNSTEMI\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Mr. ___,\nYou were admitted after you fell and...",___\n
3,3,10000764,27897940,E78,Pure hypercholesterolemia,Epistaxis\n,None\n\n,Mr. ___ is an ___ with history of AAA s/p repa...,MI after AAA repair when he was ___ y/o\nHTN\n...,___,...,NaN,ADMISSION:\n___ 08:15PM BLOOD WBC-11.3* RBC-4....,Mr. ___ is an ___ with history of AAA s/p repa...,The Preadmission Medication list is accurate a...,1. Clopidogrel 75 mg PO DAILY \n2. Acetaminoph...,Home With Service\n,Nasal fracture\nEpistaxis\nNSTEMI\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Mr. ___,\nYou were admitted after you fell and...",___\n
4,4,10000935,29541074,E78,Other and unspecified hyperlipidemia,abdominal pain and vomiting\n,"___ Exploratory laparotomy, lysis of adhesions...",The patient is a ___ year old woman s/p hyster...,PMH: \ncarcinoid tumor as above\nVitamin B12 d...,___,...,NaN,___ 04:40AM WBC-12.5*# RBC-4.46 HGB-13.6 HCT...,This ___ year old female was admitted to the h...,Albuteral MDI prn wheezes\nFlovent inhaler prn...,1. Albuterol Sulfate 90 mcg/Actuation HFA Aero...,Home\n,High grade small bowel obstruction\n\n,"Henodynamically stable, tolerating a regular d...",Please call your doctor or nurse practitioner ...,___\n


In [9]:
text_n17 = pd.read_csv("../data/text_icn17.csv")
text_n17.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000764,27897940,N17,"Acute kidney failure, unspecified",Epistaxis\n,None\n\n,Mr. ___ is an ___ with history of AAA s/p repa...,MI after AAA repair when he was ___ y/o\nHTN\n...,___,...,NaN,ADMISSION:\n___ 08:15PM BLOOD WBC-11.3* RBC-4....,Mr. ___ is an ___ with history of AAA s/p repa...,The Preadmission Medication list is accurate a...,1. Clopidogrel 75 mg PO DAILY \n2. Acetaminoph...,Home With Service\n,Nasal fracture\nEpistaxis\nNSTEMI\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Mr. ___,\nYou were admitted after you fell and...",___\n
1,1,10000980,25242409,N17,"Acute kidney failure, unspecified",DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,ADMISSION LABS:\n\n___ 02:45PM BLOOD WBC-5.1 R...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n
2,2,10001338,22119639,N17,"Acute kidney failure, unspecified",diverticulitis\n,s/p laparoscopic sigmoid colectomy \n\n,Mrs. ___ is a ___ F with history of recurrent ...,diverticulitis\nMigraines\nLeft finger celluli...,___,...,NaN,RADIOLOGY Final Report\nCT PELVIS W/CONTRAST ...,Mrs. ___ was directly admitted to ___ from Dr....,"Lexapro 10, nasonex \n",1. Escitalopram 10 mg Tablet Sig: One (1) Tabl...,NaN,Primary:\nRecurrent Diverticulitis\nPost-op wo...,"Stable\nTolerating a regular, low-residue diet...",Please call your doctor or return to the ER fo...,___\n
3,3,10001401,26840593,N17,"Acute kidney failure, unspecified","Abdominal pain, distention, nausea\n",Interventional radiology placement of abdomina...,"___ F with h/o muscle invasive bladder cancer,...","Hypertension, laparoscopic cholecystectomy, le...",___,...,NaN,___ 05:58AM BLOOD WBC-9.9 RBC-2.76* Hgb-8.2* H...,Ms. ___ was admitted to Dr. ___ service for \n...,The Preadmission Medication list is accurate a...,1. Ciprofloxacin HCl 500 mg PO Q12H Duration:...,Extended Care\n,"bladder cancer, post-operative ileus, bacterem...",Mental Status: Clear and coherent.\nLevel of C...,-Please also refer to the instructions provide...,___\n
4,4,10001401,27060146,N17,"Acute kidney failure, unspecified","subjective fevers, lethargy, and bloody drain ...","___: For the large pelvic fluid collections, C...",Ms. ___ is an ___ with PMH of hypertension and...,- Hypertension \n - s/p lap chole \n - s/p l...,___,...,NaN,ADMISSION LABS:\n===============\n___ 07:10PM ...,BRIEF SUMMARY:\n==============\n___ year old w...,The Preadmission Medication list is accurate a...,1. Ertapenem Sodium 1 g IV 1X Duration: 1 Dos...,Extended Care\n,Primary diagnosis: Pelvic fluid collection inf...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ did you come to the hospital?...",___\n


In [10]:
text_f32 = pd.read_csv("../data/text_icf32.csv")
text_f32.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000935,29541074,F32,"Depressive disorder, not elsewhere classified",abdominal pain and vomiting\n,"___ Exploratory laparotomy, lysis of adhesions...",The patient is a ___ year old woman s/p hyster...,PMH: \ncarcinoid tumor as above\nVitamin B12 d...,___,...,NaN,___ 04:40AM WBC-12.5*# RBC-4.46 HGB-13.6 HCT...,This ___ year old female was admitted to the h...,Albuteral MDI prn wheezes\nFlovent inhaler prn...,1. Albuterol Sulfate 90 mcg/Actuation HFA Aero...,Home\n,High grade small bowel obstruction\n\n,"Henodynamically stable, tolerating a regular d...",Please call your doctor or nurse practitioner ...,___\n
1,1,10000935,21738619,F32,"Depressive disorder, not elsewhere classified","nausea, vomiting, cough\n",none\n\n,"___ female, with past medical history signific...",PMH: \n# high grade SBO ___ s/p exploratory la...,___,...,NaN,___ 03:14PM ___\n___ 12:50PM URINE HOURS-RA...,___ Female with PMH significant for depression...,The Preadmission Medication list is accurate a...,1. Albuterol Inhaler ___ PUFF IH Q6H:PRN SOB \...,Home\n,Liver and Lung Mets of unkown primary\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___, \n\n___ were seen in the ED for ongo...",___\n
2,2,10000935,26381316,F32,"Depressive disorder, not elsewhere classified","Weakness, nausea/vomiting \n\n",none\n\n,This is a ___ yo f with h/o recently diagnosed...,PMH: \n# high grade SBO ___ s/p exploratory la...,___,...,NaN,ADMIT LABS:\n\n___ 04:35PM BLOOD WBC-18.7*# RB...,___ yo f with h/o recently diagnosed metastati...,The Preadmission Medication list is accurate a...,1. Albuterol Inhaler ___ PUFF IH Q6H:PRN SOB \...,Extended Care\n,Metastatic disease of unknown primary: Weaknes...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nYou were admitted to ___ for ...",___\n
3,3,10000935,25849114,F32,"Depressive disorder, not elsewhere classified",Shortness of breath\n,Percutaneous liver biopsy\n,Ms. ___ is a ___ with metastatic cancer of unk...,Per OMR:\n# metastatic cancer of unknown prima...,___,...,NaN,On admission:\n___ 12:50PM BLOOD WBC-17.9* RBC...,Ms. ___ is a ___ with metastatic cancer of unk...,The Preadmission Medication list may be inaccu...,1. BuPROPion 150 mg PO DAILY \n2. Ondansetron ...,Home With Service\n,Primary:\nMetastatic colon adenocarcinoma\n\n,Mental Status: Confused - sometimes.\nLevel of...,"Dear Ms. ___,\nIt was a pleasure taking part i...",___\n
4,4,10001186,24016413,F32,"Depressive disorder, not elsewhere classified",exposed craniotomy hardware \n,Right scalp flap with split thickness skin gra...,___ year old female with multiple prior surger...,"right parietal anaplastic astrocytoma,Cranioto...",___,...,NaN,None this admission.\n,The patient was admitted to the plastic surger...,"___: azathioprine, Pentasa, topiramate, alpraz...",1. azathioprine 75 mg Tablet Sig: One (1) Tabl...,Home With Service\n,exposed craniotomy wound Status post hardware ...,Mental Status: Clear and coherent.\nLevel of C...,-The hemovac drain should always be collapsed ...,___\n


In [11]:
text_i16 = pd.read_csv("../data/text_ici16.csv")
text_i16.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000764,27897940,I16,Unspecified essential hypertension,Epistaxis\n,None\n\n,Mr. ___ is an ___ with history of AAA s/p repa...,MI after AAA repair when he was ___ y/o\nHTN\n...,___,...,NaN,ADMISSION:\n___ 08:15PM BLOOD WBC-11.3* RBC-4....,Mr. ___ is an ___ with history of AAA s/p repa...,The Preadmission Medication list is accurate a...,1. Clopidogrel 75 mg PO DAILY \n2. Acetaminoph...,Home With Service\n,Nasal fracture\nEpistaxis\nNSTEMI\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Mr. ___,\nYou were admitted after you fell and...",___\n
1,1,10001176,23334588,I16,Unspecified essential hypertension,fever\n,none\n\n,"___ with history of morbid obesity, coronary a...",MYOCARDIAL INFARCT - INFEROPOSTERIOR \nHYPERC...,___,...,NaN,Labs on Admission:\n\n___ 10:00PM WBC-7.2 RB...,NaN,NaN,1. Escitalopram Oxalate 20 mg PO DAILY \n2. Li...,Home\n,Community Acquired Pneumonia\nDiabetes Mellitu...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted to the hos...",___\n
2,2,10001217,24597018,I16,Unspecified essential hypertension,"Left hand and face numbness, left hand weaknes...",Right parietal craniotomy for abscess incision...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis\n,___,...,NaN,___ MRI HEAD W/WO CONTRAST \nIMPRESSION: \n \...,Mrs. ___ presented to the ___ Emergency Depart...,Ibuprofen\n,1. Acetaminophen 325-650 mg PO Q6H:PRN pain \n...,Home With Service\n,Brain abscess\n\n,Mental Status: Clear and coherent.\nLevel of C...,Have a friend/family member check your incisi...,___\n
3,3,10001217,27703517,I16,Unspecified essential hypertension,Wound Infection\n,Right Craniotomy and Evacuation of Abscess on ...,Ms. ___ is a ___ y/o woman with a past medical...,Multiple sclerosis\n,___,...,NaN,MRI Brain for Operative Planning: ___\nDecreas...,Ms. ___ is a ___ y/o F who was admitted to the...,The Preadmission Medication list is accurate a...,NaN,Home With Service\n,Brain Abscess\n,Mental Status: Clear and coherent.\nLevel of C...,Your staples should stay clean and dry until ...,___\n
4,4,10001860,21441082,I16,Unspecified essential hypertension,neck pain s/p fall\n,None on this Admission\n,___ male transferred from outside hospital for...,"PMH: a. fib, colon ca, htn, copd\n \nMED: war...",___,...,NaN,NaN,Patient was admitted to the ___ ___ Surgery Se...,NaN,"1. Acetaminophen 650 mg PO Q4H:PRN pain, temp ...",Home With Service\n,C2 fracture\n,Mental Status: Clear and coherent.\nLevel of C...,You have undergone the following operation: An...,___\n


In [12]:
text_y92 = pd.read_csv("../data/text_icy92.csv")
text_y92.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000117,27988844,Y92,Sidewalk as the place of occurrence of the ext...,Left hip pain \n,"Status post left CRPP ___, ___\n\n",REASON FOR CONSULT: Femur fracture\n\nHPI: ___...,- GERD \n - Hypercholesterolemia \n - Kidney...,___,...,NaN,NaN,The patient presented to the emergency departm...,The Preadmission Medication list is accurate a...,1. Acetaminophen 1000 mg PO Q6H:PRN Pain - Mi...,Home With Service\n,Left valgus impacted femoral neck fracture\n\n,"AVSS\nNAD, A&Ox3\nLLE: Incision well approxima...",INSTRUCTIONS AFTER ORTHOPAEDIC SURGERY:\n\n- Y...,___\n
1,1,10000248,20600184,Y92,Street and highway accidents,Right flank bruising and pain s/p fall\n,None\n\n,Mr. ___ is a ___ with history of factor VIII d...,-Factor VIII deficiency (mild) \n\n,___,...,NaN,"ADMISSION, DISCHARGE, PERTINENT LABS:\n\n___ 0...",Mr. ___ is a ___ with history of mild FVIII de...,The Preadmission Medication list is accurate a...,1. Acetaminophen 1000 mg PO Q8H pain \n2. Desm...,Home\n,"PRIMARY:\n- Acute muscular hematoma, right fla...",Mental Status: Clear and coherent.\nLevel of C...,"Mr. ___,\n\nIt was our pleasure caring for you...",___\n
2,2,10000764,27897940,Y92,Accidents occurring in unspecified place,Epistaxis\n,None\n\n,Mr. ___ is an ___ with history of AAA s/p repa...,MI after AAA repair when he was ___ y/o\nHTN\n...,___,...,NaN,ADMISSION:\n___ 08:15PM BLOOD WBC-11.3* RBC-4....,Mr. ___ is an ___ with history of AAA s/p repa...,The Preadmission Medication list is accurate a...,1. Clopidogrel 75 mg PO DAILY \n2. Acetaminoph...,Home With Service\n,Nasal fracture\nEpistaxis\nNSTEMI\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Mr. ___,\nYou were admitted after you fell and...",___\n
3,3,10000980,25242409,Y92,Accidents occurring in residential institution,DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,ADMISSION LABS:\n\n___ 02:45PM BLOOD WBC-5.1 R...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n
4,4,10001401,26840593,Y92,Other specified places as the place of occurre...,"Abdominal pain, distention, nausea\n",Interventional radiology placement of abdomina...,"___ F with h/o muscle invasive bladder cancer,...","Hypertension, laparoscopic cholecystectomy, le...",___,...,NaN,___ 05:58AM BLOOD WBC-9.9 RBC-2.76* Hgb-8.2* H...,Ms. ___ was admitted to Dr. ___ service for \n...,The Preadmission Medication list is accurate a...,1. Ciprofloxacin HCl 500 mg PO Q12H Duration:...,Extended Care\n,"bladder cancer, post-operative ileus, bacterem...",Mental Status: Clear and coherent.\nLevel of C...,-Please also refer to the instructions provide...,___\n


In [13]:
text_e87 = pd.read_csv("../data/text_ice87.csv")
text_e87.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000032,22841357,E87,Hyposmolality and/or hyponatremia,abdominal fullness and discomfort\n,___ diagnostic paracentesis\n___ therapeutic p...,"___ with HIV on HAART, COPD, HCV cirrhosis com...",1. HCV Cirrhosis \n2. No history of abnormal ...,___,...,NaN,LABS ON ADMISSION:\n___ 04:10PM BLOOD ___ \n__...,"___ with HIV on HAART, HCV cirrhosis with asci...",The Preadmission Medication list is accurate a...,"1. Acetaminophen 500 mg PO Q6H:PRN pain,fever ...",Home\n,Primary: diuretic refractory ascites\nSeconda...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ was a pleasure to take care o...",___\n
1,1,10000032,29079034,E87,Hyposmolality and/or hyponatremia,altered mental status\n,none\n,"Mrs. ___ is a ___ female with HIV on HAART, CO...",- HCV Cirrhosis: genotype 3a \n- HIV: on HAAR...,___,...,NaN,ADMISSION LABS:\n=================\n___ 06:39A...,NaN,The Preadmission Medication list is accurate a...,1. Calcium Carbonate 500 mg PO BID \n2. Emtric...,Home\n,Primary:\nHypotension\nHyperkalemia\nAcute Kid...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nYou were admitted to ___ beca...",___\n
2,2,10000032,29079034,E87,Hyperpotassemia,altered mental status\n,none\n,"Mrs. ___ is a ___ female with HIV on HAART, CO...",- HCV Cirrhosis: genotype 3a \n- HIV: on HAAR...,___,...,NaN,ADMISSION LABS:\n=================\n___ 06:39A...,NaN,The Preadmission Medication list is accurate a...,1. Calcium Carbonate 500 mg PO BID \n2. Emtric...,Home\n,Primary:\nHypotension\nHyperkalemia\nAcute Kid...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nYou were admitted to ___ beca...",___\n
3,3,10000032,25742920,E87,Hyposmolality and/or hyponatremia,Abdominal pain\n,___ Paracentesis\n\n,"___ w/ HIV on HAART, COPD on 3L home O2 (thoug...",- HCV Cirrhosis: genotype 3a \n- HIV: on HAAR...,___,...,NaN,ADMISSION LABS\n==============\n___ 09:20PM BL...,NaN,NaN,1. albuterol sulfate 90 mcg/actuation inhalati...,Home With Service\n,PRIMARY DIAGNOSIS\n=================\n-Decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
4,4,10000032,25742920,E87,Hyperpotassemia,Abdominal pain\n,___ Paracentesis\n\n,"___ w/ HIV on HAART, COPD on 3L home O2 (thoug...",- HCV Cirrhosis: genotype 3a \n- HIV: on HAAR...,___,...,NaN,ADMISSION LABS\n==============\n___ 09:20PM BL...,NaN,NaN,1. albuterol sulfate 90 mcg/actuation inhalati...,Home With Service\n,PRIMARY DIAGNOSIS\n=================\n-Decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n


In [14]:
text_z85 = pd.read_csv("../data/text_icz85.csv")
text_z85.head(5)

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000084,23052089,Z85,Personal history of malignant neoplasm of pros...,Visual hallucinations\n,N/A\n\n,"___ male with ___ disease, dyslipidemia, and a...",___ disease\n___ Body Dementia \ndyslipidemia ...,___,...,NaN,ADMISSION LABS:\n===============\n___ 10:30PM ...,NaN,The Preadmission Medication list may be inaccu...,1. QUEtiapine Fumarate 25 mg PO QHS \nRX *que...,Home With Service\n,___ Dementia\n,Mental Status: Confused - sometimes.\nLevel of...,"Dear Mr. ___, \n\nIt was a privilege caring fo...",___\n
1,1,10000935,29541074,Z85,Personal history of malignant neoplasm of bron...,abdominal pain and vomiting\n,"___ Exploratory laparotomy, lysis of adhesions...",The patient is a ___ year old woman s/p hyster...,PMH: \ncarcinoid tumor as above\nVitamin B12 d...,___,...,NaN,___ 04:40AM WBC-12.5*# RBC-4.46 HGB-13.6 HCT...,This ___ year old female was admitted to the h...,Albuteral MDI prn wheezes\nFlovent inhaler prn...,1. Albuterol Sulfate 90 mcg/Actuation HFA Aero...,Home\n,High grade small bowel obstruction\n\n,"Henodynamically stable, tolerating a regular d...",Please call your doctor or nurse practitioner ...,___\n
2,2,10000935,26381316,Z85,Personal history of malignant neuroendocrine t...,"Weakness, nausea/vomiting \n\n",none\n\n,This is a ___ yo f with h/o recently diagnosed...,PMH: \n# high grade SBO ___ s/p exploratory la...,___,...,NaN,ADMIT LABS:\n\n___ 04:35PM BLOOD WBC-18.7*# RB...,___ yo f with h/o recently diagnosed metastati...,The Preadmission Medication list is accurate a...,1. Albuterol Inhaler ___ PUFF IH Q6H:PRN SOB \...,Extended Care\n,Metastatic disease of unknown primary: Weaknes...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nYou were admitted to ___ for ...",___\n
3,3,10001186,21334040,Z85,Personal history of malignant neoplasm of brain,Exposed hardware\n,Exposed hardware removal \n\n,The is a ___ year old female who had prior sur...,"right parietal anaplastic astrocytoma, Craniot...",___,...,NaN,___:\nHematology \nCOMPLETE BLOOD COUNT WBC RB...,The patient presented to the ___ neurosurgical...,The Preadmission Medication list may be inaccu...,1. ALPRAZolam 0.5 mg PO TID \n2. Azathioprine ...,Home\n,Hardware removal\n\n,Mental Status: Clear and coherent.\nLevel of C..., Please take Fluconazole 200mg once daily for...,___\n
4,4,10001186,24906418,Z85,Personal history of malignant neoplasm of brain,exposed craniotomy hardware\n,wound revision and hardware removal\n\n,This is a ___ year old female with prior surge...,"right parietal anaplastic astrocytoma,Cranioto...",___,...,NaN,CT Head\n1. No evidence of abscess formation. ...,patient presented to the ED at ___ on ___ with...,"azathioprine, Pentasa, topiramate,\nalprazolam...",1. acetaminophen 325 mg Tablet Sig: ___ Tablet...,Home\n,exposure of craniotomy hardware and infection\...,Mental Status: Clear and coherent.\nLevel of C...,Have a friend/family member check your incisi...,___\n


In [15]:
print(text_i50.shape) #(80909, 22)
print(text_e11.shape) #(108625, 22)
print(text_e78.shape)
print(text_n17.shape)
print(text_f32.shape)
print(text_i16.shape)
print(text_y92.shape)
print(text_e87.shape)
print(text_z85.shape)

(80909, 22)
(108625, 22)
(124577, 22)
(52570, 22)
(53509, 22)
(82918, 22)
(55746, 22)
(77867, 22)
(59863, 22)


In [34]:
dftexti50_e11 = pd.concat([text_i50, text_e11])

In [17]:
concat_df = pd.concat([text_i50, text_e11, text_e78, text_n17, text_f32, text_i16, text_y92, text_e87, text_z85]) #696584 rows × 22 columns

In [18]:
concat_df

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,0,10000980,29654838,I50,"Congestive heart failure, unspecified",Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,NaN,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
1,1,10000980,29654838,I50,Acute on chronic diastolic heart failure,Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,NaN,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
2,2,10000980,26913865,I50,Acute on chronic systolic heart failure,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,NaN,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
3,3,10000980,26913865,I50,"Congestive heart failure, unspecified",dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,NaN,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
4,4,10000980,24947999,I50,Acute on chronic systolic heart failure,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,...,NaN,==== ADMISSION LABS ====\n___ 04:00PM BLOOD WB...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59858,290,19997752,29452285,Z85,Personal history of malignant neoplasm of othe...,Dyspnea on exertion\n,Status post aortic valve replacement with a 21...,"___ with known h/o of aortic stenosis, initial...",- Severe aortic stenosis \n - Hypertension ...,___,...,NaN,ADMISSION LABS\n======================\n___ 06...,"___ y/o F w/ PMH aortic stenosis, HTN, childho...",The Preadmission Medication list is accurate a...,1. Aspirin EC 81 mg PO DAILY \n2. azelastin...,Extended Care\n,Status post aortic valve replacement with a 21...,"Alert and orient

In [14]:
dftexti50_e11.shape #(189534, 22)
dftexti50_e11.columns

Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'icd', 'long_title',
       'chief_complaint', 'surgical_invasive_procedure',
       'history_present_illness', 'past_medcal_history', 'social_history',
       'family_history', 'physical_exam', 'discharge', 'pertinent_results',
       'hospital_course', 'medication_on_admission', 'medication_on_discharge',
       'discharge_disposition', 'discharge_diagnosis', 'discharge_condition',
       'discharge_instruction', 'followup_instruction'],
      dtype='object')

In [35]:
df_pivot = dftexti50_e11[['subject_id', 'hadm_id', 'icd']]

In [19]:
concat_subdf = concat_df[['subject_id', 'hadm_id', 'icd']]

In [20]:
concat_subdf

,subject_id,hadm_id,icd
0,10000980,29654838,I50
1,10000980,29654838,I50
2,10000980,26913865,I50
3,10000980,26913865,I50
4,10000980,24947999,I50
...,...,...,...
59858,19997752,29452285,Z85
59859,19998562,26846592,Z85
59860,19998591,23299044,Z85
59861,19999303,27034282,Z85


In [17]:
df_pivot.tail()

,subject_id,hadm_id,icd
108620,19999287,22997012,E11
108621,19999287,20175828,E11
108622,19999379,26008899,E11
108623,19999828,29734428,E11
108624,19999828,25744818,E11


In [36]:
target1 = ((pd.get_dummies(df_pivot, columns = ['icd']).groupby(['subject_id', 'hadm_id'])
            .sum()>0).astype(int).reset_index())

In [27]:
ictarget = ((pd.get_dummies(concat_subdf, columns = ['icd']).groupby(['subject_id', 'hadm_id'])
            .sum()>0).astype(int).reset_index())

In [45]:
ictarget.to_csv('../data/ictarget.csv')

In [50]:
# ictarget = pd.read_csv('../data/ictarget.csv')

# ictarget.columns = [x.lower().replace(' ', '_') for x in ictarget.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# ictarget.to_sql('ictarget', db, if_exists = 'append', index = False) 

263941

In [51]:
query = """
SELECT Count(*) 
FROM ictarget;
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,Count(*)
0,263941


In [38]:
concat_df.shape #(696584, 22)
ictarget.shape #((263941, 11))
target1.shape #(114688, 4)
df_text_ic = concat_df.drop_duplicates(['subject_id', 'hadm_id']).merge(ictarget, how='inner', on=['subject_id', 'hadm_id'])

In [41]:
df_text_ic.shape #(263941, 31)
df_text_ic

df_text_ic.to_csv('../data/df_text_ic.csv')

In [34]:
dftexti50_e11.drop_duplicates(['subject_id', 'hadm_id']).merge(target1, how='inner', on=['subject_id', 'hadm_id'])

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction,icd_E11,icd_I50
0,0,10000980,29654838,I50,"Congestive heart failure, unspecified",Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n,1,1
1,2,10000980,26913865,I50,Acute on chronic systolic heart failure,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n,1,1
2,4,10000980,24947999,I50,Acute on chronic systolic heart failure,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n,1,1
3,6,10000980,25242409,I50,"Congestive heart failure, unspecified",DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n,1,1
4,8,10000980,25911675,I50,Acute on chronic systolic (congestive) heart f...,"fatigue, anemia\n",None\n\n,Ms. ___ is a ___ year old woman with a past me...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,Ms. ___ is a ___ year old woman with a past me...,The Preadmission Medication list may be inaccu...,1. Acetaminophen 325-650 mg PO Q6H:PRN pain or...,Home With Service\n,Primary diagnosis:\nAnemia\nCongestive heart f...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you....",___\n,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114683,556,19999287,22997012,E11,Diabetes mellitus without mention of complicat...,Shortness of breath\n,___\n1. Flexible and rigid bronchoscopy\n2. Cr...,"Ms. ___ is a ___ with a history of COPD, T1 N0...","COPD, Pulmonary function tests on ___ showed a...",___,...,Ms. ___ is a ___ with a history of poorly diff...,The Preadmission Medication list is accurate a...,1. Amlodipine 5 mg PO DAILY \n2. Bisacodyl 5 m...,Home With Service\n,Primary: \nPost-obstructive pneumonia\nRecurre...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n,1,0
114684,557,19999287,20175828,E11,Diabetes mellitus without mention of complicat...,dyspnea\n,___ - Bronchoscopy\n\n,___ hx COPD (FEV1 49% predicted ___ and poorly...,COPD Pulmonary function tests on\n___ showed a...,___,...,This is a ___ year old female with

In [30]:
target1

,subject_id,hadm_id,icd_E11,icd_I50
0,10000980,20897796,1,1
1,10000980,24947999,1,1
2,10000980,25242409,1,1
3,10000980,25911675,1,1
4,10000980,26913865,1,1
...,...,...,...,...
114683,19999287,20175828,1,0
114684,19999287,22997012,1,0
114685,19999379,26008899,1,0
114686,19999828,25744818,1,0


In [21]:
df_pivot.pivot_table(columns='icd', index = ['subject_id', 'hadm_id'])

Empty DataFrame
Columns: []
Index: [(10000980, 20897796), (10000980, 24947999), (10000980, 25242409), (10000980, 25911675), (10000980, 26913865), (10000980, 29654838), (10000980, 29659838), (10001176, 23334588), (10001667, 22672901), (10001877, 21320596), (10001877, 25679292), (10002013, 21763296), (10002013, 21975601), (10002013, 23581541), (10002013, 24760295), (10002013, 24848509), (10002013, 25442395), (10002013, 25715803), (10002013, 27574273), (10002013, 28420602), (10002013, 28603984), (10002131, 24065018), (10002155, 23822395), (10002155, 28994087), (10002221, 20195471), (10002221, 20237862), (10002221, 21729093), (10002221, 29399017), (10002428, 23473524), (10002430, 24513842), (10002430, 24648311), (10002430, 26295318), (10002430, 27218502), (10002443, 21329021), (10002495, 24982426), (10002760, 28094813), (10002976, 21640325), (10002976, 27179825), (10003019, 28836362), (10003299, 20940957), (10003299, 21476780), (10003299, 29323205), (10003400, 20214994), (10003400, 23559586), (10003400, 26467376), (10003502, 20459702), (10003502, 26475031), (10003502, 29011269), (10003637, 22082422), (10003637, 23487925), (10003637, 28484061), (10004235, 24181354), (10004322, 20356134), (10004322, 22602599), (10004322, 28755331), (10004401, 21085166), (10004401, 22869003), (10004401, 23920883), (10004401, 25753439), (10004401, 25777141), (10004401, 26164209), (10004401, 26488315), (10004401, 27939719), (10004401, 28128182), (10004401, 28154772), (10004401, 28679787), (10004401, 29988601), (10005012, 28371912), (10005236, 23384508), (10005236, 25656545), (10005368, 24631981), (10005749, 20010003), (10005749, 24015009), (10005749, 24979738), (10005817, 20626031), (10005817, 28661809), (10005858, 29569314), (10006029, 20850584), (10006029, 25426298), (10006029, 27104518), (10006029, 29107716), (10006277, 25610553), (10006457, 24934308), (10006457, 27894366), (10006513, 28504108), (10006513, 29846618), (10006580, 24159665), (10006640, 29151310), (10007174, 20280072), (10007174, 22934294), (10007462, 29383640), (10007795, 20285402), (10007795, 27962747), (10007818, 22987108), (10009614, 27624592), (10009628, 25926192), (10010058, 21955805), (10010058, 25917765), (10010058, 26359957), (10010058, 28963312), ...]

[114688 rows x 0 columns]